<a href="https://colab.research.google.com/github/Hanbin-git/kaggle/blob/main/feature_color%EB%A1%9C_%EC%A0%84%EB%9E%B5_%EC%88%98%EC%A0%952025_06_09.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import shutil
import zipfile

# 1. Drive에서 로컬로 복사 (빠름)
shutil.copy("/content/drive/MyDrive/open.zip", "/content/open.zip")

# 2. 압축 풀기
with zipfile.ZipFile("/content/open.zip", "r") as zip_ref:
    zip_ref.extractall("/content/")

# 3. 확인
!ls /content/train


컨티넨탈_10세대_2017_2019	     A_클래스_W177_2020_2025
어코드_10세대_2018_2022		     B_클래스_W246_2013_2018
1시리즈_F20_2013_2015		     뉴_스타일_코란도_C_2017_2019
1시리즈_F20_2016_2019		     프리우스_C_2018_2020
1시리즈_F40_2020_2024		     뉴_CC_2012_2016
그랜드카니발_2006_2010		     CLA_클래스_C117_2014_2019
2008_2015_2017			     CLA_클래스_C118_2020_2025
에쿠스_신형_2010_2015		     CLE_클래스_C236_2024_2025
파나메라_2010_2016		     CLS_클래스_C257_2019_2023
뉴_제타_2011_2016		     CLS_클래스_W218_2012_2017
뉴_카이엔_2011_2018		     아반떼_하이브리드_CN7_2021_2023
엑센트_신형_2011_2019		     아반떼_CN7_2021_2023
스파크_2012_2015		     더_뉴_아반떼_CN7_2023_2025
쿠퍼_컨트리맨_2012_2015		     CT6_2016_2018
올_뉴_모닝_2012_2015		     C_클래스_W204_2008_2015
아베오_2012_2016		     C_클래스_W205_2015_2021
말리부_2012_2016		     C_클래스_W206_2022_2024
뉴_티구안_2012_2016		     제네시스_DH_2014_2016
레이_2012_2017			     쏘나타_DN8_2020_2023
올란도_2012_2018		     쏘나타_디_엣지_DN8_2024_2025
더_뉴_파사트_2012_2019		     e_트론_2020_2023
트랙스_2013_2016		     E_PACE_2018_2020
콰트로포르테_2014_2016		     EQ900_2016_2018
더_뉴_아반떼_2014_2016		     E

In [2]:
import os
import glob
import torch
import timm
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
from sklearn.model_selection import StratifiedKFold


In [3]:
# train 경로
train_dir = '/content/train'

# test 경로
test_dir = '/content/test'

# sample_submission (추론 때 사용)
sample_submission_path = '/content/sample_submission.csv'


In [4]:
import torch
import gc

def show_memory_status():
    allocated = torch.cuda.memory_allocated() / (1024 ** 2)  # MB 단위
    reserved = torch.cuda.memory_reserved() / (1024 ** 2)    # MB 단위
    print(f"📊 현재 GPU 메모리 상태: Allocated = {allocated:.2f} MB | Reserved = {reserved:.2f} MB")

# 현재 CUDA 사용 가능 여부 확인
if torch.cuda.is_available():
    print("🔍 초기화 전 GPU 메모리 상태:")
    show_memory_status()

    # GPU 캐시 및 메모리 초기화
    torch.cuda.empty_cache()
    torch.cuda.ipc_collect()
    gc.collect()

    print("\n🧹 GPU 메모리 초기화 완료")
    print("🔍 초기화 후 GPU 메모리 상태:")
    show_memory_status()
else:
    print("❌ CUDA 사용 불가")


🔍 초기화 전 GPU 메모리 상태:
📊 현재 GPU 메모리 상태: Allocated = 0.00 MB | Reserved = 0.00 MB

🧹 GPU 메모리 초기화 완료
🔍 초기화 후 GPU 메모리 상태:
📊 현재 GPU 메모리 상태: Allocated = 0.00 MB | Reserved = 0.00 MB


In [5]:
import glob
import os

# ✅ 전체 JPG 파일 불러오기 (Train)
file_list = glob.glob('/content/train/*/*.jpg')

# ✅ 클래스명 추출 (폴더명 기준)
def extract_class_name_jpg(path):
    return os.path.basename(os.path.dirname(path))

class_names = sorted(set(extract_class_name_jpg(f) for f in file_list))
class_to_idx = {cls: idx for idx, cls in enumerate(class_names)}

print(f"✅ 클래스 수: {len(class_to_idx)}")  # 396개 나와야 정상

# ✅ 라벨 생성
labels = [class_to_idx[extract_class_name_jpg(f)] for f in file_list]


✅ 클래스 수: 396


In [6]:
import pandas as pd
import numpy as np
from PIL import Image
from tqdm import tqdm

# ✅ labeling 기준 (간단 rule 기반 v2 사용 예시)
def simple_pose_label_v2(img_path):
    img = Image.open(img_path).convert('RGB')
    w, h = img.size

    # Aspect ratio
    aspect_ratio = w / h

    # 중앙 crop → 중앙 밝기
    crop = img.crop((w * 0.3, h * 0.5, w * 0.7, h * 0.8))
    crop_np = np.array(crop).mean()

    # 상단 crop → 상단 밝기
    crop_top = img.crop((w * 0.3, h * 0.1, w * 0.7, h * 0.4))
    crop_top_np = np.array(crop_top).mean()

    # Simple rule v2
    if crop_np > crop_top_np + 15 and aspect_ratio > 1.1:
        return 'Rear'
    elif crop_top_np > crop_np + 15 and aspect_ratio > 0.9:
        return 'Front'
    elif 0.9 < aspect_ratio < 1.1:
        return 'Side'
    else:
        return 'Unknown'

# ✅ 전체 labeling → Train 전용
def run_pose_labeling_train(img_paths):
    records = []

    for path in tqdm(img_paths, desc='Pose labeling Train'):
        fname = os.path.basename(path)
        label = simple_pose_label_v2(path)
        # class_name 추출 (폴더명)
        class_name = extract_class_name_jpg(path)
        records.append({
            'filename': fname,
            'class': class_name,
            'pose': label
        })

    return pd.DataFrame(records)

# ✅ 실행
df_train_pose = run_pose_labeling_train(file_list)

# ✅ 저장
SAVE_CSV_PATH = '/content/drive/MyDrive/team_models/pose_labels_for_train.csv'
df_train_pose.to_csv(SAVE_CSV_PATH, index=False)

print(f"\n✅ Pose labeling (Train) 완료 → CSV 저장됨: {SAVE_CSV_PATH}")
print(df_train_pose['pose'].value_counts())


Pose labeling Train: 100%|██████████| 33137/33137 [01:36<00:00, 344.74it/s]



✅ Pose labeling (Train) 완료 → CSV 저장됨: /content/drive/MyDrive/team_models/pose_labels_for_train.csv
pose
Front      16134
Rear        8769
Unknown     7600
Side         634
Name: count, dtype: int64


In [7]:
import pandas as pd

# ✅ 파일 경로
POSE_LABEL_CSV = '/content/drive/MyDrive/team_models/pose_labels_for_train.csv'

# ✅ CSV 로드
df_pose = pd.read_csv(POSE_LABEL_CSV)

# ✅ 우선 기존 pose 컬럼 복사
df_pose['manual_pose'] = df_pose['pose']

# ✅ 적용 순서 (중요!)
# ✅ 완전 노이즈
manual_noise_list = [
    '5시리즈_G60_2024_2025_0010.jpg',
    '6시리즈_GT_G32_2018_2020_0018.jpg',
    '7시리즈_G11_2016_2018_0040.jpg',
    '911_992_2020_2024_0030.jpg',
    'E_클래스_W212_2010_2016_0022.jpg',
    'K5_2세대_2016_2018_0007.jpg',
    'F150_2004_2021_0018.jpg',
    'G_클래스_W463b_2019_2025_0030.jpg',
    'GLE_클래스_W167_2019_2024_0068.jpg',
    'Q5_FY_2021_2024_0032.jpg',
    'Q30_2017_2019_0075.jpg',
    'Q50_2014_2017_0031.jpg',
    'SM7_뉴아트_2008_2011_0053.jpg',
    'X3_G01_2022_2024_0029.jpg',
    'XF_X260_2016_2020_0023.jpg',
    '뉴_ES300h_2013_2015_0000.jpg',
    '뉴_G80_2025_2026_0042.jpg',
    '뉴_G80_2025_2026_0043.jpg',
    '뉴_SM5_임프레션_2008_2010_0033.jpg',
    '더_기아_레이_EV_2024_2025_0078.jpg',
    '더_뉴_K3_2세대_2022_2024_0001.jpg',
    '더_뉴_그랜드_스타렉스_2018_2021_0078.jpg',
    '더_뉴_그랜드_스타렉스_2018_2021_0079.jpg',
    '더_뉴_그랜드_스타렉스_2018_2021_0080.jpg',
    '더_뉴_아반떼_2014_2016_0031.jpg',
    '더_뉴_파사트_2012_2019_0067.jpg',
    '레니게이드_2019_2023_0041.jpg',
    '박스터_718_2017_2024_0011.jpg',
    '싼타페_TM_2019_2020_0009.jpg',
    '아반떼_MD_2011_2014_0081.jpg',
    '아반떼_N_2022_2023_0064.jpg',
    '익스플로러_2016_2017_0072.jpg',
    '콰트로포르테_2017_2022_0074.jpg',
    '프리우스_4세대_2019_2022_0052.jpg',
    '아반떼_N_2022_2023_0035.jpg'  # ✅ 추가로 넣으신 것!
]

# ✅ 차량 내부
manual_inside_list = [
    'E_클래스_W212_2010_2016_0069.jpg',
    'ES300h_7세대_2019_2026_0028.jpg',
    'G_클래스_W463_2009_2017_0011.jpg',
    'GLB_클래스_X247_2020_2023_0008.jpg',
    'GLS_클래스_X167_2020_2024_0013.jpg',
    'K3_2013_2015_0045.jpg',
    'K5_3세대_2020_2023_0081.jpg',
    'Q7_4M_2020_2023_0011.jpg',
    'RAV4_5세대_2019_2024_0020.jpg',
    'S_클래스_W223_2021_2025_0008.jpg',
    'S_클래스_W223_2021_2025_0071.jpg',
    'X4_F26_2015_2018_0068.jpg',
    '그랜드_체로키_WL_2021_2023_0018.jpg',
    '레이_2012_2017_0063.jpg',
    '레인지로버_5세대_2023_2024_0030.jpg',
    '레인지로버_스포츠_2세대_2018_2022_0014.jpg',
    '레인지로버_스포츠_2세대_2018_2022_0017.jpg',
    '마칸_2019_2021_0035.jpg',
    '머스탱_2015_2023_0086.jpg',
    '아반떼_MD_2011_2014_0009.jpg',
    '아반떼_MD_2011_2014_0082.jpg',
    '컨티넨탈_GT_3세대_2018_2023_0007.jpg',
    '타이칸_2021_2025_0065.jpg',
    '파나메라_2010_2016_0000.jpg',
    '파나메라_2010_2016_0036.jpg',
    '3시리즈_F30_2013_2018_0036.jpg',
    '4시리즈_F32_2014_2020_0027.jpg',
    '5시리즈_G60_2024_2025_0056.jpg',
    '7시리즈_F01_2009_2015_0029.jpg',
    '7시리즈_F01_2009_2015_0044.jpg',
    '911_992_2020_2024_0006.jpg',
    'C_클래스_W204_2008_2015_0068.jpg',
    'CLS_클래스_C257_2019_2023_0021.jpg'
]

# ✅ 트렁크 열림
manual_trunk_list = [
    'Q30_2017_2019_0074.jpg',
    '글래디에이터_JT_2020_2023_0075.jpg',
    '뉴_CC_2012_2016_0001.jpg',
    '뉴_CC_2012_2016_0002.jpg',
    '더_뉴_코나_2021_2023_0081.jpg',
    '2시리즈_액티브_투어러_U06_2022_2024_0004.jpg',
    'A8_D5_2018_2023_0084.jpg'
]

# ✅ 결과 확인
print(df_pose['manual_pose'].value_counts())

# ✅ 저장
SAVE_MANUAL_CSV = '/content/drive/MyDrive/team_models/pose_labels_for_train_manual_applied.csv'
df_pose.to_csv(SAVE_MANUAL_CSV, index=False)

print(f"\n✅ 자동 적용 완료 → CSV 저장됨: {SAVE_MANUAL_CSV}")


manual_pose
Front      16134
Rear        8769
Unknown     7600
Side         634
Name: count, dtype: int64

✅ 자동 적용 완료 → CSV 저장됨: /content/drive/MyDrive/team_models/pose_labels_for_train_manual_applied.csv


In [8]:
import pandas as pd
import glob
import os

# ✅ 준비된 Pose Label CSV 경로
POSE_CSV_PATH = '/content/drive/MyDrive/team_models/pose_labels_for_train_manual_applied.csv'

# ✅ 기존 pose label 로드
df_pose = pd.read_csv(POSE_CSV_PATH)

# ✅ 현재 file_list 로드 (train image list)
train_file_list = glob.glob('/content/train/*/*.jpg')

# ✅ "파일명" 기준으로 DataFrame 만들기
df_train_files = pd.DataFrame({
    'filepath': train_file_list,
    'filename': [os.path.basename(p) for p in train_file_list],
    'class': [os.path.basename(os.path.dirname(p)) for p in train_file_list]
})

# ✅ Merge → pose 붙이기
df_train_merged = pd.merge(df_train_files, df_pose[['filename', 'manual_pose']], on='filename', how='left')

# ✅ Null 체크
assert df_train_merged['manual_pose'].isnull().sum() == 0, "🚨 pose 매칭 안된 이미지 있음!"

print(f"✅ Pose merge 완료 → 총 {len(df_train_merged)}개")

# ✅ One-hot Encoding (추천 사용)
df_pose_ohe = pd.get_dummies(df_train_merged['manual_pose'], prefix='pose')

# ✅ 최종 DF
df_train_final = pd.concat([df_train_merged, df_pose_ohe], axis=1)

# ✅ 저장
SAVE_TRAIN_FINAL = '/content/drive/MyDrive/team_models/train_with_pose_feature.csv'
df_train_final.to_csv(SAVE_TRAIN_FINAL, index=False)

print(f"\n✅ 최종 Train + Pose feature 저장 완료 → {SAVE_TRAIN_FINAL}")
print(f"👉 One-hot columns: {df_pose_ohe.columns.tolist()}")


✅ Pose merge 완료 → 총 33137개

✅ 최종 Train + Pose feature 저장 완료 → /content/drive/MyDrive/team_models/train_with_pose_feature.csv
👉 One-hot columns: ['pose_Front', 'pose_Rear', 'pose_Side', 'pose_Unknown']


In [9]:
import torch
from torch.utils.data import Dataset
from PIL import Image
import cv2
import numpy as np
import os

class CarImagePoseDataset(Dataset):
    def __init__(self, file_list, class_to_idx, df_pose, transform=None,
                 use_aspect=False, use_color=False, use_pose=False):

        self.file_list = file_list
        self.class_to_idx = class_to_idx
        self.df_pose = df_pose.set_index('filename')  # 빠른 lookup 위해 index 설정
        self.transform = transform
        self.use_aspect = use_aspect
        self.use_color = use_color
        self.use_pose = use_pose

    def __len__(self):
        return len(self.file_list)

    def __getitem__(self, idx):
        path = self.file_list[idx]

        # Load image
        image_pil = Image.open(path).convert("RGB")

        # Aspect Ratio
        width, height = image_pil.size
        aspect_ratio = np.array([width / height], dtype=np.float32)

        # Color Mean
        image_cv2 = cv2.cvtColor(np.array(image_pil), cv2.COLOR_RGB2BGR)
        color_mean = image_cv2.mean(axis=(0, 1))
        color_mean = color_mean[::-1]
        color_mean = np.array(color_mean / 255.0, dtype=np.float32)

        # Transform
        if self.transform:
            image = self.transform(image_pil)
        else:
            image = transforms.ToTensor()(image_pil)

        # Label
        class_name = os.path.basename(os.path.dirname(path))
        label = self.class_to_idx[class_name]

        # Filename
        fname = os.path.basename(path)

        # Pose feature (4-dim vector)
        pose_vec = np.array([
            self.df_pose.loc[fname, 'pose_Front'],
            self.df_pose.loc[fname, 'pose_Rear'],
            self.df_pose.loc[fname, 'pose_Side'],
            self.df_pose.loc[fname, 'pose_Unknown']
        ], dtype=np.float32)

        # Return modes
        features = []

        if self.use_aspect:
            features.append(torch.tensor(aspect_ratio))

        if self.use_color:
            features.append(torch.tensor(color_mean))

        if self.use_pose:
            features.append(torch.tensor(pose_vec))

        if features:
            return image, *features, label
        else:
            return image, label


In [10]:
import torch.nn as nn
import timm
import torch

class CustomModel(nn.Module):
    def __init__(self, use_aspect, use_color, use_pose, num_classes):
        super(CustomModel, self).__init__()

        self.use_aspect = use_aspect
        self.use_color = use_color
        self.use_pose = use_pose

        # ✅ EfficientNet-B5 backbone
        self.backbone = timm.create_model('efficientnet_b5', pretrained=True, num_classes=0)  # feature extractor
        backbone_out_features = self.backbone.num_features

        # ✅ Meta feature dimension 계산
        meta_features_dim = 0
        if self.use_aspect:
            meta_features_dim += 1  # aspect ratio 1개
        if self.use_color:
            meta_features_dim += 3  # color_mean (R, G, B) 3개
        if self.use_pose:
            meta_features_dim += 4  # pose one-hot (Front, Rear, Side, Unknown)

        # ✅ Classifier
        self.classifier = nn.Sequential(
            nn.Linear(backbone_out_features + meta_features_dim, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(512, num_classes)
        )

    def forward(self, image, aspect_ratio=None, color_mean=None, pose_feature=None):
        # ✅ EfficientNet feature
        x = self.backbone(image)

        # ✅ Meta features concat
        aux_list = []

        if self.use_aspect:
            aux_list.append(aspect_ratio)

        if self.use_color:
            aux_list.append(color_mean)

        if self.use_pose:
            aux_list.append(pose_feature)

        if aux_list:
            aux_features = torch.cat(aux_list, dim=1)
            x = torch.cat([x, aux_features], dim=1)

        # ✅ Final classifier
        out = self.classifier(x)
        return out


In [11]:
from torchvision import transforms

# EfficientNet-B5 권장 입력 크기 (456x456)
IMG_SIZE = 456

train_transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(15),
    transforms.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.3, hue=0.1),
    transforms.RandomAffine(degrees=0, translate=(0.1, 0.1), scale=(0.9, 1.1)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

val_transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])


In [12]:
import torch
from torch.utils.data import Dataset
from PIL import Image
import cv2
import numpy as np
import os

class CarImageDataset(Dataset):
    def __init__(self, file_list, class_to_idx, df_pose, transform=None, use_aspect=False, use_color=False, use_pose=False):
        self.file_list = file_list
        self.class_to_idx = class_to_idx
        self.df_pose = df_pose
        self.transform = transform
        self.use_aspect = use_aspect
        self.use_color = use_color
        self.use_pose = use_pose

    def __len__(self):
        return len(self.file_list)

    def __getitem__(self, idx):
        path = self.file_list[idx]

        # 🚗 Load Image
        image_pil = Image.open(path).convert("RGB")

        # 🚗 Aspect Ratio
        width, height = image_pil.size
        aspect_ratio = np.array([width / height], dtype=np.float32)

        # 🚗 Color Mean
        image_cv2 = cv2.cvtColor(np.array(image_pil), cv2.COLOR_RGB2BGR)
        color_mean = image_cv2.mean(axis=(0, 1))
        color_mean = color_mean[::-1]
        color_mean = np.array(color_mean / 255.0, dtype=np.float32)

        # 🚗 Pose One-hot → df_pose lookup
        fname = os.path.basename(path)
        row = self.df_pose[self.df_pose['filename'] == fname]

        if len(row) == 0:
            pose_onehot = np.array([0, 0, 0, 1], dtype=np.float32)  # Unknown fallback
        else:
            pose = row['pose'].values[0]
            pose_onehot = np.zeros(4, dtype=np.float32)
            if pose == 'Front':
                pose_onehot[0] = 1
            elif pose == 'Rear':
                pose_onehot[1] = 1
            elif pose == 'Side':
                pose_onehot[2] = 1
            else:
                pose_onehot[3] = 1  # Unknown

        # 🚗 Transform
        if self.transform:
            image = self.transform(image_pil)
        else:
            image = transforms.ToTensor()(image_pil)

        # 🚗 Label
        class_name = os.path.basename(os.path.dirname(path))
        label = self.class_to_idx[class_name]

        # 🚗 Return
        meta_list = []

        if self.use_aspect:
            meta_list.append(torch.tensor(aspect_ratio))
        if self.use_color:
            meta_list.append(torch.tensor(color_mean))
        if self.use_pose:
            meta_list.append(torch.tensor(pose_onehot))

        if meta_list:
            return image, *meta_list, label
        else:
            return image, label


In [13]:
import torch

# ✅ 디바이스 설정 (GPU 우선)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"✅ Device 사용: {device}")


✅ Device 사용: cuda


In [ ]:
from sklearn.model_selection import StratifiedKFold
import copy
from tqdm import tqdm
import torch

# ✅ StratifiedKFold 정의
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# ✅ 실험명
EXPERIMENT = "C_pose"

use_aspect = False
use_color = True
use_pose = True

print(f"\n🚀 실험 설정: {EXPERIMENT} (Aspect={use_aspect}, Color={use_color}, Pose={use_pose})\n")

# ✅ 5-Fold 루프 시작
for fold, (train_idx, val_idx) in enumerate(skf.split(file_list, labels)):
    print(f"\n==============================")
    print(f"🔁 Fold {fold + 1} / 5")
    print(f"==============================\n")

    # ✅ Fold별 split
    train_files = [file_list[i] for i in train_idx]
    val_files = [file_list[i] for i in val_idx]

    # ✅ Fold별 Dataset & DataLoader
    train_dataset = CarImageDataset(train_files, class_to_idx, df_pose, train_transform, use_aspect, use_color, use_pose)
    val_dataset = CarImageDataset(val_files, class_to_idx, df_pose, val_transform, use_aspect, use_color, use_pose)

    train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True, num_workers=4, pin_memory=True, persistent_workers=True, prefetch_factor=2)
    val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False, num_workers=4, pin_memory=True, persistent_workers=True, prefetch_factor=2)

    # ✅ Fold별 model / criterion / optimizer 초기화
    model = CustomModel(use_aspect=use_aspect, use_color=use_color, use_pose=use_pose, num_classes=396)
    model = model.to(device)

    criterion = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4, weight_decay=1e-4)

    # ✅ EarlyStopping 변수 초기화
    best_val_loss = float('inf')
    patience = 3
    patience_counter = 0
    best_model_wts = copy.deepcopy(model.state_dict())

    # ✅ Epoch 루프
    for epoch in range(1, 31):
        print(f"\n📌 Fold {fold+1} | Epoch {epoch}")

        # === 학습 ===
        model.train()
        train_loss = 0.0
        train_correct = 0

        loop = tqdm(train_loader, desc=f"Train Fold {fold+1}", leave=False)
        for batch in loop:
            X = batch[0].to(device)
            meta_list = batch[1:-1]
            y = batch[-1].to(device)

            # Meta 준비
            meta_args = {}
            cnt = 0
            if use_aspect:
                meta_args['aspect_ratio'] = meta_list[cnt].to(device)
                cnt += 1
            if use_color:
                meta_args['color_mean'] = meta_list[cnt].to(device)
                cnt += 1
            if use_pose:
                meta_args['pose_feature'] = meta_list[cnt].to(device)
                cnt += 1

            outputs = model(X, **meta_args)

            loss = criterion(outputs, y)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            train_loss += loss.item() * X.size(0)
            train_correct += (outputs.argmax(1) == y).sum().item()
            loop.set_postfix(loss=loss.item())

        train_loss /= len(train_loader.dataset)
        train_acc = train_correct / len(train_loader.dataset)

        # === 검증 ===
        model.eval()
        val_loss = 0.0
        val_correct = 0

        val_loop = tqdm(val_loader, desc=f"Valid Fold {fold+1}", leave=False)
        with torch.no_grad():
            for batch in val_loop:
                X = batch[0].to(device)
                meta_list = batch[1:-1]
                y = batch[-1].to(device)

                meta_args = {}
                cnt = 0
                if use_aspect:
                    meta_args['aspect_ratio'] = meta_list[cnt].to(device)
                    cnt += 1
                if use_color:
                    meta_args['color_mean'] = meta_list[cnt].to(device)
                    cnt += 1
                if use_pose:
                    meta_args['pose_feature'] = meta_list[cnt].to(device)
                    cnt += 1

                outputs = model(X, **meta_args)

                loss = criterion(outputs, y)
                val_loss += loss.item() * X.size(0)
                val_correct += (outputs.argmax(1) == y).sum().item()
                val_loop.set_postfix(loss=loss.item())

        val_loss /= len(val_loader.dataset)
        val_acc = val_correct / len(val_loader.dataset)

        # === 로그 출력 ===
        print(f"✅ Fold {fold+1} | Epoch {epoch} | Train Loss: {train_loss:.4f} | Acc: {train_acc:.4f}")
        print(f"✅ Fold {fold+1} | Epoch {epoch} | Val   Loss: {val_loss:.4f} | Acc: {val_acc:.4f}")

        # === EarlyStopping ===
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            best_model_wts = copy.deepcopy(model.state_dict())
            save_path = f"/content/drive/MyDrive/team_models/EffNetB5_{EXPERIMENT}_fold{fold+1}.pth"
            torch.save(model.state_dict(), save_path)
            print(f"📦 Best model saved for Fold {fold+1}!")
            patience_counter = 0
        else:
            patience_counter += 1
            print(f"⚠️ EarlyStopping patience: {patience_counter}/{patience}")
            if patience_counter >= patience:
                print("⛔ Early stopping triggered.")
                break

    # ✅ Fold 끝나고 Best 모델 로드
    model.load_state_dict(best_model_wts)
    print(f"✅ Fold {fold+1} Best model loaded.\n")



🚀 실험 설정: C_pose (Aspect=False, Color=True, Pose=True)


🔁 Fold 1 / 5



/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(



📌 Fold 1 | Epoch 1


✅ Fold 1 | Epoch 1 | Train Loss: 3.2506 | Acc: 0.4787
✅ Fold 1 | Epoch 1 | Val   Loss: 0.6897 | Acc: 0.8612
📦 Best model saved for Fold 1!

📌 Fold 1 | Epoch 2


✅ Fold 1 | Epoch 2 | Train Loss: 0.5051 | Acc: 0.8977
✅ Fold 1 | Epoch 2 | Val   Loss: 0.2809 | Acc: 0.9200
📦 Best model saved for Fold 1!

📌 Fold 1 | Epoch 3


...

✅ Fold 3 | Epoch 11 | Train Loss: 0.1110 | Acc: 0.9639
✅ Fold 3 | Epoch 11 | Val   Loss: 0.1842 | Acc: 0.9513
⚠️ EarlyStopping patience: 3/3
⛔ Early stopping triggered.
✅ Fold 3 Best model loaded.


🔁 Fold 4 / 5


📌 Fold 4 | Epoch 1


✅ Fold 4 | Epoch 1 | Train Loss: 3.2348 | Acc: 0.4814
✅ Fold 4 | Epoch 1 | Val   Loss: 0.7080 | Acc: 0.8524
📦 Best model saved for Fold 4!

📌 Fold 4 | Epoch 2


✅ Fold 4 | Epoch 2 | Train Loss: 0.5055 | Acc: 0.8971
✅ Fold 4 | Epoch 2 | Val   Loss: 0.2851 | Acc: 0.9188
📦 Best model saved for Fold 4!

📌 Fold 4 | Epoch 3


✅ Fold 4 | Epoch 3 | Train Loss: 0.2639 | Acc: 0.9279
✅ Fold 4 | Epoch 3 | Val   Loss: 0.2400 | Acc: 0.9301
📦 Best model saved for Fold 4!

📌 Fold 4 | Epoch 4


Train Fold 4:  99%|█████████▊| 1635/1657 [06:18<00:05,  4.33it/s, loss=0.256]

In [ ]:
# ✅ 설정
import pandas as pd
import numpy as np
import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import cv2
import os
from tqdm import tqdm
import timm

# ✅ 하이퍼파라미터
IMG_SIZE = 456
NUM_CLASSES = 396
TEST_DIR = "/content/test"
SAMPLE_SUB_PATH = "/content/sample_submission.csv"
exp_name = "C"

# ✅ 디바이스 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"✅ Using device: {device}")

# ✅ 샘플 제출 파일
sample = pd.read_csv(SAMPLE_SUB_PATH)
column_names = sample.columns.tolist()[1:]  # 'ID' 제외

# ✅ Transform 고정
test_transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

# ✅ Test Dataset (Color만 사용)
class TestDataset_ColorOnly(Dataset):
    def __init__(self, img_root, transform=None):
        self.file_list = sorted([os.path.join(img_root, f) for f in os.listdir(img_root) if f.endswith('.jpg')])
        self.transform = transform

    def __len__(self):
        return len(self.file_list)

    def __getitem__(self, idx):
        path = self.file_list[idx]
        image_pil = Image.open(path).convert("RGB")

        # Color Mean
        image_cv2 = cv2.cvtColor(np.array(image_pil), cv2.COLOR_RGB2BGR)
        color_mean = image_cv2.mean(axis=(0, 1))
        color_mean = color_mean[::-1]
        color_mean = np.array(color_mean / 255.0, dtype=np.float32)

        if self.transform:
            image = self.transform(image_pil)
        else:
            image = transforms.ToTensor()(image_pil)

        fname = os.path.basename(path).replace(".jpg", "")
        return image, torch.tensor(color_mean), fname

# ✅ CustomModel (전략 C용)
class CustomModel(nn.Module):
    def __init__(self, use_aspect, use_color, use_pose, num_classes):
        super(CustomModel, self).__init__()

        self.use_aspect = use_aspect
        self.use_color = use_color
        self.use_pose = use_pose

        self.backbone = timm.create_model('efficientnet_b5', pretrained=True, num_classes=0)
        backbone_out_features = self.backbone.num_features

        meta_features_dim = 0
        if self.use_aspect:
            meta_features_dim += 1
        if self.use_color:
            meta_features_dim += 3
        if self.use_pose:
            meta_features_dim += 4

        self.classifier = nn.Sequential(
            nn.Linear(backbone_out_features + meta_features_dim, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(512, num_classes)
        )

    def forward(self, image, aspect_ratio=None, color_mean=None, pose_feature=None):
        x = self.backbone(image)

        aux_list = []
        if self.use_aspect:
            aux_list.append(aspect_ratio)
        if self.use_color:
            aux_list.append(color_mean)
        if self.use_pose:
            aux_list.append(pose_feature)

        if aux_list:
            aux_features = torch.cat(aux_list, dim=1)
            x = torch.cat([x, aux_features], dim=1)

        out = self.classifier(x)
        return out

# ✅ 모델 경로
FOLD_MODEL_PATHS = [
    f"/content/drive/MyDrive/team_models/EffNetB5_{exp_name}_fold1.pth",
    f"/content/drive/MyDrive/team_models/EffNetB5_{exp_name}_fold2.pth",
    f"/content/drive/MyDrive/team_models/EffNetB5_{exp_name}_fold3.pth",
    f"/content/drive/MyDrive/team_models/EffNetB5_{exp_name}_fold4.pth",
    f"/content/drive/MyDrive/team_models/EffNetB5_{exp_name}_fold5.pth",
]

# ✅ DataLoader
test_dataset = TestDataset_ColorOnly(TEST_DIR, test_transform)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=4, pin_memory=True)

# ✅ 앙상블 결과
ensemble_outputs = []

# ✅ 추론 루프
for fold_idx, model_path in enumerate(FOLD_MODEL_PATHS):
    print(f"\n🚀 Inference with Fold {fold_idx + 1} Model: {model_path}")

    model = CustomModel(use_aspect=False, use_color=True, use_pose=False, num_classes=NUM_CLASSES)
    model.load_state_dict(torch.load(model_path, map_location=device))
    model = model.to(device)
    model.eval()

    fold_probs = []

    with torch.no_grad():
        for imgs, color_mean, fnames in tqdm(test_loader, desc=f"🔍 Fold {fold_idx + 1} Inference"):
            imgs, color_mean = imgs.to(device), color_mean.to(device)

            outputs = model(imgs, color_mean=color_mean)
            probs = F.softmax(outputs, dim=1)
            fold_probs.append(probs.cpu().numpy())

    fold_probs = np.concatenate(fold_probs, axis=0)
    ensemble_outputs.append(fold_probs)

# ✅ 앙상블 평균
ensemble_outputs = np.stack(ensemble_outputs, axis=0)
mean_outputs = np.mean(ensemble_outputs, axis=0)

# ✅ 결과 저장
results = []
for idx, path in enumerate(test_dataset.file_list):
    fname = os.path.basename(path).replace(".jpg", "")
    row = {"ID": fname}
    row.update({class_name: mean_outputs[idx, i] for i, class_name in enumerate(column_names)})
    results.append(row)

submission_df = pd.DataFrame(results)
submission_df = submission_df[["ID"] + column_names]

# ✅ 저장
SAVE_SUB_PATH = f"/content/drive/MyDrive/team_models/submission_fold5_ensemble_{exp_name}_IMG{IMG_SIZE}.csv"
submission_df.to_csv(SAVE_SUB_PATH, index=False)

print(f"\n✅ 최종 제출용 CSV 저장 완료: {SAVE_SUB_PATH}")
